# Intro
在AI的数据部分，主要处理以下任务：
1. 数据收集：收集样本和对应标签（可能是整个AI流程中最麻烦的一部分）
2. 数据划分：将数据划分为训练集，测试集，验证集
3. 数据读取：数据读取要求给定一个索引，能返回一个样本及标签，即index => （data，label)这样的一个映射，在torch中，对应的实现模块为DataLoader，其中包含一个Sampler和DataSet，其中Sampler负责生成索引，DataSet根据索引返回样本及标签
4. 数据预处理：将数据输入转化为模型要求的输入格式，对应于torch的transform模块。

# 数据收集
这一部分工作，一般借助爬虫等工具完成，还有数据清理等工作比较琐碎，不予介绍，在这里我们使用RMB_data

通常而言，数据收集完后交付的数据集一般是如下形式，即不同文件夹对应不同label的数据
```shell
-- XXX_data
    | -- label_1
        | -- a.png
        | -- b.png
        | -- c.png
    | -- label_2
        | -- x.png
        | -- y.png
        | -- z.png
```

其他形式例如：xxx.csv等

# 数据划分
数据划分的任务就是划分数据为train, valid, test三个集合，划分如下：
```shell
-- XXX_split
    | -- train
        | -- label_1
            | -- a.png
            | -- b.png
        | -- label_2
    | -- valid
        | -- label_1
        | -- label_2
    | -- test
        | -- label_1
        | -- label_2
```

In [1]:
import os 
import random 
import shutil

In [8]:
def makedir(new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)
        
random.seed(1)

dataset_dir = os.path.join("data", "RMB_data")
split_dir = os.path.join("data", "rmb_split")

train_dir = os.path.join(split_dir, "train")
valid_dir = os.path.join(split_dir, "valid")
test_dir = os.path.join(split_dir, "test")

train_pct = 0.8
valid_pct = 0.1
test_pct = 0.1

for root, dirs, files in os.walk(dataset_dir):
    for sub_dir in dirs:
        imgs = os.listdir(os.path.join(root, sub_dir))
        imgs = list(filter(lambda x: x.endswith('.jpg'), imgs))
        random.shuffle(imgs)
        img_count = len(imgs)
        train_point = int(img_count * train_pct)
        valid_point = int(img_count * (train_pct + valid_pct))
        for i in range(img_count):
            if i < train_point:
                out_dir = os.path.join(train_dir, sub_dir)
            elif i < valid_point:
                out_dir = os.path.join(valid_dir, sub_dir)
            else:
                out_dir = os.path.join(test_dir, sub_dir)
            makedir(out_dir)
            target_path = os.path.join(out_dir, imgs[i])
            src_path = os.path.join(dataset_dir, sub_dir, imgs[i])
            shutil.copy(src_path, target_path)
        print('Class: {}, train: {}, valid: {}, test: {}'.format(sub_dir, train_point, valid_point-train_point, img_count-valid_point))

Class: 100, train: 80, valid: 10, test: 10
Class: 1, train: 80, valid: 10, test: 10


# 数据读取
在torch中，数据读取范式为DataLoader，在实现DataLoader之前，需要先实现Dataset，Dataset根据索引返回样本及标签（在__getitem__中实现)

Dataset: indices => (data, label)

DataLoader实现如下：
DataLoader(Dataset, batch_size=BATCH_SIZE, shuffle=True/False)

在使用时：
```python
for inputs, labels in train_loader:
    ....
```

In [12]:
import os
import random
from PIL import Image 
from torch.utils.data import Dataset

In [14]:
random.seed(1)
rmb_label = {"1": 0, "100": 1}

class RMBDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        """
            self.data_info为一个list，因此可以通过它知道其有多少个元素
            在这里是[(img_path, int(label))]
        """
        self.data_info = self.get_img_info(data_dir)
        self.transform = transform
    
    def __getitem__(self, index):
        img_path, label = self.data_info[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label
    
    def __len__(self):
        return len(self.data_info)
    
    @staticmethod
    def get_img_info(data_dir):
        data_info = list()
        for root, dirs, _ in os.walk(data_dir):
            for sub_dir in dirs:
                img_names = os.listdir(os.path.join(root, sub_dir))
                img_names = list(filter(lambda x: x.endswith('.jpg'), img_names))
                for i in range(len(img_names)):
                    img_name = img_names[i]
                    img_path = os.path.join(root, sub_dir, img_name)
                    label = rmb_label[sub_dir]
                    data_info.append((img_path, int(label)))
        return data_info

# 数据预处理
transform模块使用如下：
```python
# 设置训练集的数据增强和转化
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

# 设置验证集的数据增强和转化，不需要 RandomCrop
valid_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

# 构建MyDataset实例
train_data = RMBDataset(data_dir=train_dir, transform=train_transform)
valid_data = RMBDataset(data_dir=valid_dir, transform=valid_transform)
```